Chargement des paquets
---

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import datetime as dt

from pandas_profiling import ProfileReport
import dataframe_image as dfi


import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.colors as clrs

from sklearn import preprocessing #ajout du numéro 


%matplotlib inline

import time
import os

In [2]:
big_t_start0 = time.time()

---

Chargement des jeux de données
---

In [3]:
df_ath1 = pd.read_csv("../data/raw/bdd_ath1_vf.csv")
df_ath2 = pd.read_csv("../data/raw/bdd_ath2_vf.csv")

df_date_id_ath1 = pd.read_csv("../data/raw/dates_id_ath1_vf.csv")
df_date_id_ath2 = pd.read_csv("../data/raw/dates_id_ath2_vf.csv")

df_labels_ath1_vf = pd.read_csv("../data/raw/rpe_labels_ath1_vf.csv")
df_labels_ath2_vf = pd.read_csv("../data/raw/rpe_labels_ath2_vf.csv")

df_morning_ath1_vf = pd.read_csv("../data/raw/morning_ath1_vf.csv")
df_morning_ath2_vf = pd.read_csv("../data/raw/morning_ath2_vf.csv")

df_phases_ath1 = pd.read_csv("../data/raw/phases_ath1.csv")
df_phases_ath2 = pd.read_csv("../data/raw/phases_ath2.csv")


In [4]:
print("athlete 1")
print("df_ath1.shape             ", df_ath1.shape)
print("df_date_id_ath1.shape     ", df_date_id_ath1.shape)
print("df_labels_ath1_vf.shape   ", df_labels_ath1_vf.shape)
print("df_morning_ath1_vf.shape  ", df_morning_ath1_vf.shape)
print("df_phases_ath1.shape      ", df_phases_ath1.shape)

print(" \n ----------------------------------------- \n ")

print("athlete 2")
print("df_ath2.shape             ", df_ath2.shape)
print("df_date_id_ath2.shape     ", df_date_id_ath2.shape)
print("df_labels_ath2_vf.shape   ", df_labels_ath2_vf.shape)
print("df_morning_ath2_vf.shape  ", df_morning_ath2_vf.shape)
print("df_phases_ath2.shape      ", df_phases_ath2.shape)

athlete 1
df_ath1.shape              (921334, 24)
df_date_id_ath1.shape      (141, 3)
df_labels_ath1_vf.shape    (142, 5)
df_morning_ath1_vf.shape   (150, 15)
df_phases_ath1.shape       (371, 5)
 
 ----------------------------------------- 
 
athlete 2
df_ath2.shape              (767623, 24)
df_date_id_ath2.shape      (129, 3)
df_labels_ath2_vf.shape    (129, 5)
df_morning_ath2_vf.shape   (20, 15)
df_phases_ath2.shape       (97, 5)


---

# Formatage des jeux de données

Les dates sont initialement au format string dans les données. On les convertit en time objects de Python pour pouvoir les filtrer et jouer avec.  

Ensuite on nettoie un peu les colonnes en supprimant celles qui ne servent à rien. (typiquement **Steps.per.Minute** et **Swim.Cadence**)  

Puis on fait un encodage particulier pour numéroter par entiers croissants les jours d'entraînement d'une part et les types d'entraînement dans une même journée d'autre part. À un jour donné, pour un type d'entraînement, les points de mesure sont déjà numérotés par entiers croissants dans la colonne **tps** que l'on renommera également.  
**Pour être clair**, le dataframe final comportera trois colonnes de valeurs entières:
1. La première représente le nombre de jours d'entraînement dans la saison (e.g. 1 pour le premier entraînement, 120 pour le 120ème entraînement).
1. La deuxième colonne représente le nombre d'entraînements dans la même journée ; elle est majoritairement valorisée à 1, mais il arrive qu'une athlète fasse au plus 4 types d'entraînement différents dans la même journée (e.g. échauffement, endurance, intensité et récupération).
1. Et enfin, la dernière colonne représente le numéro du point de mesure au cours d'un entraînement donné ; cette valeur va 1 jusqu'à \~10000 pour les entraînements les plus longs, elle est réinitialisée à 1 quand on change de type d'entraînement dans une même journée.   

In [5]:
big_t_start1 = time.time()

In [6]:
def df_create_right_timestamp(Date, Time):
    return dt.datetime.combine(Date, Time)

t_start = time.time()

df_ath1['Start.Time'] =  pd.to_datetime(df_ath1['Start.Time'], format='%Y-%m-%d %H:%M:%S')
df_ath1['date_only'] = df_ath1['Start.Time'].dt.date                                           #C> - Cette colonne ne sert à rien: contient seulement 4 dates
df_ath1['time_only'] = df_ath1['Start.Time'].dt.time

df_ath1['Date'] =  pd.to_datetime(df_ath1['Date'], format='%Y-%m-%d').dt.date
df_phases_ath1['date'] = pd.to_datetime(df_phases_ath1['date'], format="%Y-%m-%d").dt.date

#C>-----

df_ath2['Start.Time'] =  pd.to_datetime(df_ath2['Start.Time'], format='%Y-%m-%d %H:%M:%S')
df_ath2['date_only'] = df_ath2['Start.Time'].dt.date                                           #C> - Cette colonne ne sert à rien: contient seulement 4 dates
df_ath2['time_only'] = df_ath2['Start.Time'].dt.time

df_ath2['Date'] =  pd.to_datetime(df_ath2['Date'], format='%Y-%m-%d').dt.date
df_phases_ath2['date'] = pd.to_datetime(df_phases_ath2['date'], format="%Y-%m-%d").dt.date

#C>-----

df_ath1['Timestamp'] = df_ath1.apply(lambda row: df_create_right_timestamp(row["Date"], row["time_only"]), axis=1)
df_ath2['Timestamp'] = df_ath2.apply(lambda row: df_create_right_timestamp(row["Date"], row["time_only"]), axis=1)

df_ath1 = df_ath1.sort_values(['Timestamp'], ascending=[True])
df_ath2 = df_ath2.sort_values(['Timestamp'], ascending=[True])
# df_ath1 = df_ath1.sort_values(['Date', 'time_only'], ascending=[True, True])
# df_ath2 = df_ath2.sort_values(['Date', 'time_only'], ascending=[True, True])
print(f"Time taken to rearrange and sort both dataframes: {time.time()-t_start}")


Time taken to rearrange and sort both dataframes: 19.8934223651886


In [7]:
t_start = time.time()

print(f"Before merge: len(df_ath1) = {len(df_ath1)}; len(df_ath2) = {len(df_ath2)}")
df_ath1 = pd.merge(df_ath1, df_labels_ath1_vf[['id','type']], on='id', how='left')
df_phases_ath1.rename(columns = {'date':'Date'}, inplace = True)
df_ath1 = pd.merge(df_ath1, df_phases_ath1[['Date','day_cycle', 'phase']], on='Date', how='left')

df_ath2 = pd.merge(df_ath2, df_labels_ath2_vf[['id','type']], on='id', how='left')
df_phases_ath2.rename(columns={"date": "Date"}, inplace = True)
df_ath2 = pd.merge(df_ath2, df_phases_ath2[['Date','day_cycle', 'phase']], on='Date', how='left')
print(f"After merge: len(df_ath1) = {len(df_ath1)}; len(df_ath2) = {len(df_ath2)}")

print(f"\nTime taken {time.time()-t_start}")

Before merge: len(df_ath1) = 921334; len(df_ath2) = 767623
After merge: len(df_ath1) = 928649; len(df_ath2) = 767623

Time taken 0.743018627166748


In [8]:
del df_ath1["Steps.per.Minute"]
del df_ath1["Swim.Cadence"]
del df_ath1["Start.Time"]
del df_ath1["date_only"]
del df_ath1["max_hr"]

del df_ath2["Steps.per.Minute"]
del df_ath2["Swim.Cadence"]
del df_ath2["Start.Time"]
del df_ath2["date_only"]
del df_ath2["max_hr"]

---

Création des trois colonnes de valeurs entières pour le numéro des jours, le numéro des entraînements et le numéro des points.

In [9]:
def encode_day_training_point(df_tmp):
    list_of_dates = np.sort(df_tmp['Date'].unique())
    list_dates_int = [(date, ctr, 1) for ctr, date in enumerate(list_of_dates, start=1)]
    dict_dates_int = dict(zip(np.arange(0,120,1), list_dates_int))
    df_dates_int = pd.DataFrame.from_dict(dict_dates_int, orient='index', columns=['Date', 'day_index', 'training_index'])

    df_tmp = pd.merge(df_tmp, df_dates_int, on='Date', how='left')

    for date in list_of_dates:
        Series_tps = df_tmp[df_tmp['Date'] == date]['tps']
        tps1_indices = Series_tps[Series_tps == 1].index
    
        if len(tps1_indices) == 2 :
            idx_tps1 = tps1_indices[1]
            idx_tps_max = Series_tps.loc[idx_tps1:].idxmax()
            df_tmp.loc[idx_tps1:idx_tps_max, ['training_index']] = 2
        
        if len(tps1_indices) > 2 :
            for i in range(1, len(tps1_indices)-1):
                idx_tps1 = tps1_indices[i]
                idx_tps_max = tps1_indices[i+1]-1
                df_tmp.loc[idx_tps1:idx_tps_max, ['training_index']] = i+1
            
            idx_tps1 = idx_tps_max+1
            idx_tps_max = Series_tps.loc[idx_tps1:].idxmax()
            df_tmp.loc[idx_tps1:idx_tps_max, ['training_index']] = i+2

    return df_tmp

# %timeit encode_day_training_point(df_ath1)
t_start = time.time()

df_ath1 = encode_day_training_point(df_ath1)
df_ath2 = encode_day_training_point(df_ath2)

print(f"Time taken {time.time()-t_start}")

Time taken 9.227909326553345


Ici, pour les types d'entraînement (endurance, intensité etc.) on remplace les nan, qui sont des float, par des chaînes de caractère 'nan'.

In [10]:
def split_trainings(df_tmp, lower_threshold=np.timedelta64(30,'m')):
    df_tmp = df_tmp.sort_values(['Timestamp'], ascending=[True])

    time_delta = df_tmp['Timestamp'].diff()
    date_delta = df_tmp['Date'].diff()

    list_index_breaks = df_tmp[(time_delta > lower_threshold) & (date_delta < one_day)].index.tolist()
    print(f"Number of times there are no data for more than {lower_threshold}: {len(list_index_breaks)}")

    for idx in list_index_breaks :
        date_ncur, date_nbef = df_tmp.loc[idx, "Date"], df_tmp.loc[idx-1, "Date"]
        if date_ncur == date_nbef :
            train_idx_ncur, train_idx_nbef = df_tmp.loc[idx, "training_index"], df_tmp.loc[idx-1, "training_index"] 
            tps_ncur, tps_nbef = df_tmp.loc[idx, "tps"], df_tmp.loc[idx-1, "tps"]

            if train_idx_ncur == train_idx_nbef+1 :
                print(f"On date {date_ncur}, already two distinct trainings with time-delta > {lower_threshold} ({df_tmp.loc[idx-1, 'type']}:{train_idx_nbef}, {df_tmp.loc[idx, 'type']}:{train_idx_ncur}) \n")

            elif train_idx_ncur == train_idx_nbef and tps_ncur == tps_nbef+1 :
                print(df_tmp.loc[idx-1, ["Timestamp", "Date", "time_only", "training_index", "tps", "type"]].tolist())
                print(df_tmp.loc[idx, ["Timestamp", "Date", "time_only", "training_index", "tps", "type"]].tolist())
                print(" ")
                df_loc = df_tmp.iloc[idx:]
                df_loc = df_loc[df_loc["Date"] == date_ncur]
                
                if len(df_loc["training_index"].unique()) == 1:
                    idx_max = df_loc['tps'].idxmax()

                    df_tmp.loc[idx:idx_max, 'tps'] = np.arange(1, idx_max-idx+2, 1)
                    df_tmp.loc[idx:idx_max, 'training_index'] = train_idx_nbef+1

                else :
                    for ctr, group in enumerate(df_loc.groupby('training_index'), start=1):
                        idx_min, idx_max = group[1]['tps'].idxmin(), group[1]['tps'].idxmax()
                        df_tmp.loc[idx_min:idx_max, 'training_index'] = group[0]+1

            else :
                print("Something you haven't thought of.")
                print(df_tmp.loc[idx-1, ["Timestamp", "Date", "time_only", "training_index", "tps", "type"]].tolist())
                print(df_tmp.loc[idx, ["Timestamp", "Date", "time_only", "training_index", "tps", "type"]].tolist())
                print(" ")

        else :
            print("Nothing...", df_tmp.loc[idx, "Date"], df_tmp.loc[idx-1, "Date"])

    print(" ")

    return df_tmp

In [11]:
t_start = time.time()

one_second, thirty_minute, one_day = np.timedelta64(1,'s'), np.timedelta64(30,'m'), np.timedelta64(1, 'D')

print("Athlète 1:")
df_ath1 = split_trainings(df_ath1, lower_threshold=thirty_minute)
print("------------------------\nAthlète 2:")
df_ath2 = split_trainings(df_ath2, lower_threshold=thirty_minute)

print(f"Time taken: {time.time()-t_start}")

Athlète 1:
Number of times there are no data for more than 30 minutes: 8
[Timestamp('2021-05-03 17:24:05'), datetime.date(2021, 5, 3), datetime.time(17, 24, 5), 1, 1007, 'endurance']
[Timestamp('2021-05-03 17:57:35'), datetime.date(2021, 5, 3), datetime.time(17, 57, 35), 1, 1008, 'endurance']
 
On date 2021-05-15, already two distinct trainings with time-delta > 30 minutes (endurance:1, endurance:2) 

[Timestamp('2021-06-05 15:08:11'), datetime.date(2021, 6, 5), datetime.time(15, 8, 11), 1, 2748, 'intensity']
[Timestamp('2021-06-05 16:03:07'), datetime.date(2021, 6, 5), datetime.time(16, 3, 7), 1, 2749, 'intensity']
 
[Timestamp('2021-06-11 17:59:53'), datetime.date(2021, 6, 11), datetime.time(17, 59, 53), 1, 1002, 'intensity']
[Timestamp('2021-06-11 18:37:50'), datetime.date(2021, 6, 11), datetime.time(18, 37, 50), 1, 1003, 'intensity']
 
On date 2021-06-30, already two distinct trainings with time-delta > 30 minutes (test:1, endurance:2) 

[Timestamp('2021-07-30 15:00:33'), datetime.

In [12]:
def create_training_counter(df_tmp):
    ctr = 0
    listou = []
    for date, d1 in df_tmp.groupby("Date"):
        for training_idx, d2 in d1.groupby("training_index"):
            ctr += 1
            listou.append({"Date":date, "training_index":training_idx, "training_counter":ctr})
    small_df = pd.DataFrame(listou)

    return df_tmp.merge(small_df, how='left', on=["Date","training_index"])

t_start = time.time()

df_ath1 = create_training_counter(df_ath1)
df_ath2 = create_training_counter(df_ath2)

print(f"Time taken: {time.time()-t_start}")

Time taken: 1.1137769222259521


# On renomme les colonnes

In [13]:
df_ath1 = df_ath1.rename(columns={'Duree':'duree','Date': 'date', 'type': 'type_ent','Activite':'activite','Heart.Rate':'heart_rate','Power..Watts.':'power_watts',
                             'Speed..meters.per.sec.':'speed_meters_per_sec','Bike.Cadence':"bike_cadence",'Total.Distance..meters.':'distance_cumul',
                             'Latitude.in.Degree':'latitude','Longitude.in.Degree':'longitude','Elevation..meters.':'altitude',
                             'Air.Temperature..celsius.':'temperature','Kcal':"kcal_cst","Distance":"dist_totale_cst"})

df_ath2 = df_ath2.rename(columns={'Duree':'duree','Date': 'date', 'type': 'type_ent','Activite':'activite','Heart.Rate':'heart_rate','Power..Watts.':'power_watts',
                             'Speed..meters.per.sec.':'speed_meters_per_sec','Bike.Cadence':"bike_cadence",'Total.Distance..meters.':'distance_cumul',
                             'Latitude.in.Degree':'latitude','Longitude.in.Degree':'longitude','Elevation..meters.':'altitude',
                             'Air.Temperature..celsius.':'temperature','Kcal':"kcal_cst","Distance":"dist_totale_cst"})
print(" ")

# On réordonne les colonnes

In [14]:
col_list = ['Timestamp', 'date', 'time_only', 'training_counter', 'day_index', 'training_index', 'tps', 'type_ent', 'day_cycle', 'phase', 'activite',
            'latitude', 'longitude', 'altitude', 'temperature', 'heart_rate', 'speed_meters_per_sec', 'power_watts', 'bike_cadence', 'distance_cumul',
            'duree', 'kcal_cst', 'id', 'dist_totale_cst', 'mean_power', 'max_power', 'mean_hr', 'name']

df_ath1 = df_ath1[col_list]
df_ath2 = df_ath2[col_list]

In [15]:
print(f"Athlete 1: # colonnes {len(df_ath1.columns)} - colonnes {df_ath1.columns.tolist()}")
print(" ----------------------------------------- ")
print(f"Athlete 2: # colonnes {len(df_ath2.columns)} - colonnes {df_ath2.columns.tolist()}")

Athlete 1: # colonnes 28 - colonnes ['Timestamp', 'date', 'time_only', 'training_counter', 'day_index', 'training_index', 'tps', 'type_ent', 'day_cycle', 'phase', 'activite', 'latitude', 'longitude', 'altitude', 'temperature', 'heart_rate', 'speed_meters_per_sec', 'power_watts', 'bike_cadence', 'distance_cumul', 'duree', 'kcal_cst', 'id', 'dist_totale_cst', 'mean_power', 'max_power', 'mean_hr', 'name']
 ----------------------------------------- 
Athlete 2: # colonnes 28 - colonnes ['Timestamp', 'date', 'time_only', 'training_counter', 'day_index', 'training_index', 'tps', 'type_ent', 'day_cycle', 'phase', 'activite', 'latitude', 'longitude', 'altitude', 'temperature', 'heart_rate', 'speed_meters_per_sec', 'power_watts', 'bike_cadence', 'distance_cumul', 'duree', 'kcal_cst', 'id', 'dist_totale_cst', 'mean_power', 'max_power', 'mean_hr', 'name']


## On remplit les trous dans les séries temporelle du même entraînement

In [16]:
def resampling(df): #fonction qui bouche les trous 
    df['add_tps']=0
    df_fin = pd.DataFrame()
    for day, d1 in df.groupby('day_index'):
        for training, d2 in d1.groupby('training_index'):
            timestamp_start = min(d2.Timestamp)
            timestamp_end = max(d2.Timestamp)
            diff=timestamp_end-timestamp_start
            longueur_trou = int(diff / dt.timedelta(seconds=1))
            list_timestamp = [timestamp_start+dt.timedelta(seconds=n) for n in range(longueur_trou+1)]
            list_tps=[tps+1 for tps in range(longueur_trou+1)]
            
            df_add = pd.DataFrame(list(zip(list_timestamp,list_tps)),columns=['Timestamp','tps'])
            
            df_add['date']=d2['date'].iloc[0]
            df_add['training_counter']=d2['training_counter'].iloc[0]
            df_add['day_index'] = day
            df_add['training_index'] = training
            df_add['type_ent']=d2['type_ent'].iloc[0]

            df_add['day_cycle']=d2['day_cycle'].iloc[0]
            df_add['phase']=d2['phase'].iloc[0]
            df_add['activite']=d2['activite'].iloc[0]

            df_add['duree']=d2['duree'].iloc[0]
            df_add['kcal_cst']=d2['kcal_cst'].iloc[0]
            df_add['id']=d2['id'].iloc[0]
            df_add['dist_totale_cst']=d2['dist_totale_cst'].iloc[0]
            df_add['mean_power']=d2['mean_power'].iloc[0]
            df_add['max_power']=d2['max_power'].iloc[0]
            df_add['mean_hr']=d2['mean_hr'].iloc[0]
            df_add['name']=d2['name'].iloc[0]
            
            df_fin = pd.concat([df_fin,df_add])
            
    df_fin.reset_index(drop=True,inplace=True)
    df["Timestamp"] = pd.to_datetime(df["Timestamp"])

    df_final = df_fin.merge(df[['Timestamp', 'latitude', 'longitude', 'altitude', 'temperature',
                                'heart_rate', 'speed_meters_per_sec', 'power_watts', 'bike_cadence',
                                'distance_cumul', 'add_tps']], on='Timestamp',how="left")



    df[df['add_tps']!=0]=1
    return df_final
            
            

In [17]:
t_start = time.time()

df_ath1_full_date = resampling(df_ath1)
df_ath2_full_date = resampling(df_ath2)

print(f"Time taken: {time.time()-t_start}")

Time taken: 17.934672355651855


--- 

In [18]:
df_ath1_full_date.to_pickle("../data/formatted/df_ath1_full.pkl")
df_ath2_full_date.to_pickle("../data/formatted/df_ath2_full.pkl")

In [19]:
print(f"Total time taken to format the data: {time.time()-big_t_start0}")

Total time taken to format the data: 56.05195641517639
